In [68]:
import pandas as pd
import geopy # to retreive the postal code from latitude and longitude
import matplotlib.pyplot as plt


In [69]:
df = pd.read_csv('MCI_Data.csv')
df.head()

,X,Y,OBJECTID,EVENT_UNIQUE_ID,REPORT_DATE,OCC_DATE,REPORT_YEAR,REPORT_MONTH,REPORT_DAY,REPORT_DOY,...,UCR_CODE,UCR_EXT,OFFENCE,MCI_CATEGORY,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,LONG_WGS84,LAT_WGS84
0,-79.425896,43.757346,1,GO-20141262074,2014/01/01 05:00:00+00,1998/06/01 04:00:00+00,2014,January,1,1,...,1480,110,Administering Noxious Thing,Assault,38,Lansing-Westgate,38,Lansing-Westgate (38),-79.425896,43.757346
1,-79.350229,43.646293,2,GO-20141260701,2014/01/01 05:00:00+00,2014/01/01 05:00:00+00,2014,January,1,1,...,2120,200,B&E,Break and Enter,70,South Riverdale,70,South Riverdale (70),-79.350229,43.646293
2,-79.376497,43.666423,3,GO-20141260889,2014/01/01 05:00:00+00,2014/01/01 05:00:00+00,2014,January,1,1,...,1430,100,Assault,Assault,74,North St.James Town,74,North St.James Town (74),-79.376497,43.666423
3,-85.488744,0.000000,4,GO-20141260973,2014/01/01 05:00:00+00,2014/01/01 05:00:00+00,2014,January,1,1,...,2130,210,Theft Over,Theft Over,NSA,NSA,NSA,NSA,-85.488744,0.000000
4,-79.344839,43.678946,5,GO-20141261050,2014/01/01 05:00:00+00,2014/01/01 05:00:00+00,2014,January,1,1,...,1430,100,Assault,Assault,69,Blake-Jones,66,Danforth (66),-79.344839,43.678946


In [70]:
df.columns

Index(['X', 'Y', 'OBJECTID', 'EVENT_UNIQUE_ID', 'REPORT_DATE', 'OCC_DATE',
       'REPORT_YEAR', 'REPORT_MONTH', 'REPORT_DAY', 'REPORT_DOY', 'REPORT_DOW',
       'REPORT_HOUR', 'OCC_YEAR', 'OCC_MONTH', 'OCC_DAY', 'OCC_DOY', 'OCC_DOW',
       'OCC_HOUR', 'DIVISION', 'LOCATION_TYPE', 'PREMISES_TYPE', 'UCR_CODE',
       'UCR_EXT', 'OFFENCE', 'MCI_CATEGORY', 'HOOD_158', 'NEIGHBOURHOOD_158',
       'HOOD_140', 'NEIGHBOURHOOD_140', 'LONG_WGS84', 'LAT_WGS84'],
      dtype='object')

In [86]:
df.shape

(323296, 31)

In [71]:
df.info()
# Clean from column #18 till #30

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 323296 entries, 0 to 323295
Data columns (total 31 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   X                  323296 non-null  float64
 1   Y                  323296 non-null  float64
 2   OBJECTID           323296 non-null  int64  
 3   EVENT_UNIQUE_ID    323296 non-null  object 
 4   REPORT_DATE        323296 non-null  object 
 5   OCC_DATE           323296 non-null  object 
 6   REPORT_YEAR        323296 non-null  int64  
 7   REPORT_MONTH       323296 non-null  object 
 8   REPORT_DAY         323296 non-null  int64  
 9   REPORT_DOY         323296 non-null  int64  
 10  REPORT_DOW         323296 non-null  object 
 11  REPORT_HOUR        323296 non-null  int64  
 12  OCC_YEAR           323191 non-null  float64
 13  OCC_MONTH          323191 non-null  object 
 14  OCC_DAY            323191 non-null  float64
 15  OCC_DOY            323191 non-null  float64
 16  OC

In [72]:
df.rename(columns={'LONG_WGS84': 'Longtiude', 'LAT_WGS84': 'Latitude'}, inplace=True) # change latitude and longtiude column's name

In [73]:
df.isnull().sum()

X                      0
Y                      0
OBJECTID               0
EVENT_UNIQUE_ID        0
REPORT_DATE            0
OCC_DATE               0
REPORT_YEAR            0
REPORT_MONTH           0
REPORT_DAY             0
REPORT_DOY             0
REPORT_DOW             0
REPORT_HOUR            0
OCC_YEAR             105
OCC_MONTH            105
OCC_DAY              105
OCC_DOY              105
OCC_DOW              105
OCC_HOUR               0
DIVISION               0
LOCATION_TYPE          0
PREMISES_TYPE          0
UCR_CODE               0
UCR_EXT                0
OFFENCE                0
MCI_CATEGORY           0
HOOD_158               0
NEIGHBOURHOOD_158      0
HOOD_140               0
NEIGHBOURHOOD_140      0
Longtiude              0
Latitude               0
dtype: int64

In [74]:
# Trim strings values
df['DIVISION'] = df['DIVISION'].str.strip()
df['LOCATION_TYPE'] = df['LOCATION_TYPE'].str.strip()
df['PREMISES_TYPE'] = df['PREMISES_TYPE'].str.strip()
df['OFFENCE'] = df['OFFENCE'].str.strip()
df['MCI_CATEGORY'] = df['MCI_CATEGORY'].str.strip()
df['NEIGHBOURHOOD_158'] = df['NEIGHBOURHOOD_158'].str.strip()
df['NEIGHBOURHOOD_140'] = df['NEIGHBOURHOOD_140'].str.strip()

In [76]:
df.LOCATION_TYPE.unique()

array(['Apartment (Rooming House, Condo)',
       'Commercial Dwelling Unit (Hotel, Motel, B & B, Short Term Rental)',
       'Streets, Roads, Highways (Bicycle Path, Private Road)',
       'Bar / Restaurant',
       'Other Commercial / Corporate Places (For Profit, Warehouse, Corp. Bldg',
       'Go Train', 'Single Home, House (Attach Garage, Cottage, Mobile)',
       'Hospital / Institutions / Medical Facilities (Clinic, Dentist, Morgue)',
       'Retirement Home',
       'Gas Station (Self, Full, Attached Convenience)',
       'Open Areas (Lakes, Parks, Rivers)',
       'Parking Lots (Apt., Commercial Or Non-Commercial)',
       'Ttc Subway Station', 'Ttc Subway Train',
       "Other Non Commercial / Corporate Places (Non-Profit, Gov'T, Firehall)",
       'Schools During Supervised Activity', 'Unknown',
       'Bank And Other Financial Institutions (Money Mart, Tsx)',
       'Convenience Stores', 'Jails / Detention Centres',
       'Group Homes (Non-Profit, Halfway House, Social Age

In [78]:
df.PREMISES_TYPE.unique()

array(['Apartment', 'Commercial', 'Outside', 'Transit', 'House', 'Other',
       'Educational'], dtype=object)

In [77]:
df['LOCATION_TYPE'].value_counts()

LOCATION_TYPE
Apartment (Rooming House, Condo)                                          77032
Single Home, House (Attach Garage, Cottage, Mobile)                       58263
Streets, Roads, Highways (Bicycle Path, Private Road)                     51108
Other Commercial / Corporate Places (For Profit, Warehouse, Corp. Bldg    37226
Parking Lots (Apt., Commercial Or Non-Commercial)                         27699
Bar / Restaurant                                                          12693
Open Areas (Lakes, Parks, Rivers)                                          6663
Schools During Supervised Activity                                         4818
Convenience Stores                                                         4084
Ttc Subway Station                                                         3991
Commercial Dwelling Unit (Hotel, Motel, B & B, Short Term Rental)          3642
Private Property Structure (Pool, Shed, Detached Garage)                   3026
Hospital / Institutions / 

In [80]:
#Create PREMISES_TYPE for Bar/Resturant, Parking Lots and Open Areas
df['PREMISES_TYPE'] = df.apply(lambda x:x['LOCATION_TYPE'] if x['LOCATION_TYPE']=='Bar / Restaurant' else x['PREMISES_TYPE'], axis=1)
df['PREMISES_TYPE'] = df.apply(lambda x:'Parking Lots' if x['LOCATION_TYPE']=='Parking Lots (Apt., Commercial Or Non-Commercial)' else x['PREMISES_TYPE'], axis=1)
df['PREMISES_TYPE'] = df.apply(lambda x:'Open Areas' if x['LOCATION_TYPE']=='Open Areas (Lakes, Parks, Rivers)' else x['PREMISES_TYPE'], axis=1)

In [81]:
df.PREMISES_TYPE.unique()

array(['Apartment', 'Commercial', 'Outside', 'Bar / Restaurant',
       'Transit', 'House', 'Other', 'Open Areas', 'Parking Lots',
       'Educational'], dtype=object)

In [82]:
df.columns

Index(['X', 'Y', 'OBJECTID', 'EVENT_UNIQUE_ID', 'REPORT_DATE', 'OCC_DATE',
       'REPORT_YEAR', 'REPORT_MONTH', 'REPORT_DAY', 'REPORT_DOY', 'REPORT_DOW',
       'REPORT_HOUR', 'OCC_YEAR', 'OCC_MONTH', 'OCC_DAY', 'OCC_DOY', 'OCC_DOW',
       'OCC_HOUR', 'DIVISION', 'LOCATION_TYPE', 'PREMISES_TYPE', 'UCR_CODE',
       'UCR_EXT', 'OFFENCE', 'MCI_CATEGORY', 'HOOD_158', 'NEIGHBOURHOOD_158',
       'HOOD_140', 'NEIGHBOURHOOD_140', 'Longtiude', 'Latitude'],
      dtype='object')

In [84]:
df_18_30 = df[['DIVISION', 'LOCATION_TYPE', 'PREMISES_TYPE', 'UCR_CODE',
       'UCR_EXT', 'OFFENCE', 'MCI_CATEGORY', 'HOOD_158', 'NEIGHBOURHOOD_158',
       'HOOD_140', 'NEIGHBOURHOOD_140', 'Longtiude', 'Latitude' ]]

In [85]:
df_18_30.head()

,DIVISION,LOCATION_TYPE,PREMISES_TYPE,UCR_CODE,UCR_EXT,OFFENCE,MCI_CATEGORY,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,Longtiude,Latitude
0,D32,"Apartment (Rooming House, Condo)",Apartment,1480,110,Administering Noxious Thing,Assault,38,Lansing-Westgate,38,Lansing-Westgate (38),-79.425896,43.757346
1,D51,"Commercial Dwelling Unit (Hotel, Motel, B & B,...",Commercial,2120,200,B&E,Break and Enter,70,South Riverdale,70,South Riverdale (70),-79.350229,43.646293
2,D51,"Apartment (Rooming House, Condo)",Apartment,1430,100,Assault,Assault,74,North St.James Town,74,North St.James Town (74),-79.376497,43.666423
3,NSA,"Streets, Roads, Highways (Bicycle Path, Privat...",Outside,2130,210,Theft Over,Theft Over,NSA,NSA,NSA,NSA,-85.488744,0.000000
4,D54,"Streets, Roads, Highways (Bicycle Path, Privat...",Outside,1430,100,Assault,Assault,69,Blake-Jones,66,Danforth (66),-79.344839,43.678946


In [88]:
df_18_30.to_csv('MCIClean1830.csv')

In [79]:
def get_postalCode(df,lat, long): # function to convert latitude and longtiude to postal code
    while True:
        if lat == 0:
            continue
        geo = geopy.Nominatim(user_agent='1234')
        location = geo.reverse((df[lat], df[long]))
        postalCode = location[0].split(',')[-2]
            
        return postalCode 
#df['PostalCode']=df.apply(get_postalCode, axis=1, long='Longtiude',lat='Latitude')